In [1]:
import tabula
from tabula import read_pdf
from tabulate import tabulate
from pypdf import PdfReader
import pdfplumber
import tika
from tika import parser
import pypdfium2 as pdfium
tika.initVM()
import sys
import json
import logging
import re
import zipfile
from pprint import pprint

import pandas as pd

from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import (SdkException, ServiceApiException, ServiceUsageException)
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import ExtractElementType
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import ExtractPDFOptions

C:\Users\pytho\AppData\Roaming\Python\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from Utilities.pdf_utils  import *

In [3]:
def getData(file_path):

    zipfile = extract_pdf_and_zip(file_path)

    df_list , json_data = zip_processor(zipfile)
    # print(df_list)

    work_dict = json_getter(json_data)
#     pprint(work_dict)
    return df_list, work_dict

file_path = "C:\\noc\\SD%20Payor%20Scraping\\YourReport2.pdf"

dfs, work_d = getData(file_path)
# print(work_d)

Successfully extracted information from PDF. Printing H1 Headers:

['structuredData.json', 'tables/fileoutpart7.xlsx', 'tables/fileoutpart21.xlsx', 'tables/fileoutpart35.xlsx', 'tables/fileoutpart33.xlsx', 'tables/fileoutpart22.xlsx', 'tables/fileoutpart28.xlsx', 'tables/fileoutpart23.xlsx', 'tables/fileoutpart9.xlsx', 'tables/fileoutpart24.xlsx', 'tables/fileoutpart8.xlsx', 'tables/fileoutpart18.xlsx', 'tables/fileoutpart13.xlsx', 'tables/fileoutpart34.xlsx', 'tables/fileoutpart12.xlsx', 'tables/fileoutpart10.xlsx', 'tables/fileoutpart25.xlsx', 'tables/fileoutpart30.xlsx', 'tables/fileoutpart20.xlsx', 'tables/fileoutpart17.xlsx', 'tables/fileoutpart27.xlsx', 'tables/fileoutpart15.xlsx', 'tables/fileoutpart37.xlsx', 'tables/fileoutpart31.xlsx', 'tables/fileoutpart0.xlsx', 'tables/fileoutpart11.xlsx', 'tables/fileoutpart19.xlsx', 'tables/fileoutpart3.xlsx', 'tables/fileoutpart1.xlsx', 'tables/fileoutpart2.xlsx', 'tables/fileoutpart16.xlsx', 'tables/fileoutpart29.xlsx', 'tables/fileoutpa

In [4]:
dfs[0]

,SERVICE DATES,SERVICE CODE,ALTERNATE BENEFIT CODE,TOOTH NUM.,SURFACE,NUM. SVCS,SUBMITTED CHARGES,ALLOWABLE AMOUNT/QPA,COPAY AMOUNT,NOT SEE PAYABLE REMARKS,DEDUCTIBLE,CO INSURANCE,PATIENT RESP,PAYABLE AMOUNT
0,03/29/23,D1206,NaN,01,NaN,NaN,67.00,61.00,NaN,6.00 1,NaN,NaN,67.00,0.00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.00 2,NaN,NaN,NaN,NaN
2,03/29/23,D1120,NaN,01,NaN,1.0,96.00,90.00,NaN,6.00 1,NaN,NaN,6.00,90.00
3,03/29/23,D0230,NaN,01,NaN,1.0,37.00,33.00,NaN,4.00 1,33.00,NaN,37.00,0.00
4,03/29/23,D0220,NaN,01,NaN,1.0,42.00,39.00,NaN,3.00 1,17.00,4.40,24.40,17.60
5,03/29/23,D0120,NaN,01,NaN,1.0,75.00,70.00,NaN,5.00 1,NaN,NaN,5.00,70.00
6,TOTALS,NaN,NaN,NaN,NaN,NaN,317.00,293.00,NaN,85.00,50.00,4.40,139.40,177.60
7,ISSUED AMT: $177.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfs_list = [d for d in dfs if d.shape[1] >= 10]
print(len(dfs_list))
print(dfs_list[0].shape)
# dfs_list[1]

11
(8, 14)


In [16]:
dfs_list[1]

,Aetna Life Insurance Company Network Status: Out-of-Network,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,SERVICE DATES,SERVICE CODE,ALTERNATE BENEFIT CODE,TOOTH NUM.,SURFACE,NUM. SVCS,SUBMITTED CHARGES,ALLOWABLE AMOUNT/QPA,COPAY AMOUNT,NOT SEE PAYABLE REMARKS,DEDUCTIBLE,CO INSURANCE,PATIENT RESP,PAYABLE AMOUNT
1,04/03/23,D1206,NaN,01,NaN,1.0,67.00,66.00,NaN,1.00 1,NaN,NaN,1.00,66.00
2,04/03/23,D1120,NaN,01,NaN,1.0,96.00,95.00,NaN,NaN,NaN,NaN,NaN,96.00
3,04/03/23,D0274,NaN,01,NaN,1.0,91.00,90.00,NaN,NaN,NaN,NaN,NaN,91.00
4,04/03/23,D0230,NaN,01,NaN,1.0,37.00,36.00,NaN,NaN,NaN,7.40,7.40,29.60
5,04/03/23,D0220,NaN,01,NaN,1.0,42.00,41.00,NaN,NaN,NaN,8.40,8.40,33.60
6,04/03/23,D0120,NaN,01,NaN,1.0,75.00,74.00,NaN,NaN,NaN,NaN,NaN,75.00
7,TOTALS,NaN,NaN,NaN,NaN,NaN,408.00,402.00,NaN,1.00,NaN,15.80,16.80,391.20


In [7]:
def change_header(dfs_list):
    d_list = []
    word = 'SERVICE'
    for df in dfs_list:
        cols = list(df.columns)
        for c in cols[0:1]:
            if word in c:
                break
            else:
                header_row_index = 0
                header_row = df.iloc[header_row_index]
                df = df.set_axis(header_row, axis='columns')
                df.drop(index=header_row_index,inplace=True)
                break
        d_list.append(df)
    return d_list   


In [8]:
dfs_ch_list = change_header(dfs_list)

In [9]:
print(dfs_ch_list[3].shape)
print(dfs_ch_list[0])
print(dfs_ch_list[0].columns)

(1, 14)
         SERVICE DATES  SERVICE CODE   ALTERNATE BENEFIT CODE  TOOTH NUM.   \
0             03/29/23         D1206                       NaN         01    
1                   NaN           NaN                      NaN         NaN   
2             03/29/23         D1120                       NaN         01    
3             03/29/23         D0230                       NaN         01    
4             03/29/23         D0220                       NaN         01    
5             03/29/23         D0120                       NaN         01    
6               TOTALS            NaN                      NaN         NaN   
7  ISSUED AMT: $177.60            NaN                      NaN         NaN   

   SURFACE  NUM. SVCS  SUBMITTED CHARGES  ALLOWABLE AMOUNT/QPA   \
0       NaN        NaN             67.00                 61.00    
1       NaN        NaN                NaN                   NaN   
2       NaN       1.0              96.00                 90.00    
3       NaN       1.0

In [10]:
df_header_13 = ['ServiceDate', 'ProcCode', 'AlternateBenefitCode', 'ToothNumber', 'SurfaceNumSvcs', 'SubmittedCharges', 'Allowableamount/QPA', 'CopayAmount', 'ContractualObligations', 'Deductible', 'CoInsurance', 'PatientResp', 'PayableAmount']

dfs1 = []
for d in dfs_ch_list:
    print(d.shape)
    if d.shape[1] == 13:
        d.columns = df_header_13
    dfs1.append(d)

(8, 14)
(7, 14)
(7, 14)
(1, 14)
(8, 14)
(5, 13)
(7, 14)
(7, 14)
(7, 14)
(7, 14)
(7, 13)


In [11]:
# Camel Case of header
dfs1 = []
for d in dfs_ch_list:
    columns_list = d.columns.tolist()
    for i, c in enumerate(columns_list):
        cs = c.split()
        c = ''.join(c.title() for c in cs)
        columns_list[i] = c

In [12]:
dfs1 = []
for d in dfs_ch_list:
    columns_list = d.columns.tolist()
    for i, c in enumerate(columns_list):
        if 'SERVICE DATES' in c:
            columns_list[i] = 'ServiceDate'
        if 'SERVICE CODE' in c:
            columns_list[i] = 'ProcCode'
        if 'ALTERNATE BENEFIT' in c:
            columns_list[i] = 'AlternateBenefitCode'
        if 'TOOTH' in c:
            columns_list[i] = 'ToothNumber'
        if 'SURFACE' in c:
            columns_list[i] = 'Surface' 
        if 'SUBMITTED CHARGES' in c:
            columns_list[i] = 'SubmittedCharges' 
        if 'ALLOWABLE AMOUNT' in c:
            columns_list[i] = 'Allowableamount/QPA'     
        if 'COPAY AMOUNT' in c:
            columns_list[i] = 'CopayAmount'
        if 'NOT SEE PAYABLE REMARKS' in c:
            columns_list[i] = 'ContractualObligations' 
        if 'DEDUCTIBLE' in c:
            columns_list[i] = 'Deductible'  
        if 'CO INSURANCE' in c:
            columns_list[i] = 'CoInsurance'
        if 'PATIENT RESP' in c:
            columns_list[i] = 'PatientResp'
        if 'PAYABLE AMOUNT' in c:
            columns_list[i] = 'PayableAmount'
        if 'NUM. SVCS' in c :
            columns_list[i] = 'RemarkCodes'
        if 'NOT PAYABLE' in c :
            columns_list[i] = 'NotPayable'
        if 'SEE REMARKS' in c :
            columns_list[i] = 'SeeRemarks'     
            
                    
    d.columns = columns_list
    dfs1.append(d)
            

In [13]:
t_list = []
for df in dfs1:
#     df.drop(df[df['ServiceDate'] == 'TOTALS '].index, inplace=True)
#     df.drop(df[df['ServiceDate'].str.contains('ISSUED')].index, inplace=True)
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.fillna('')
    
    if 'NumSVCS' in df.columns:
        df.rename(columns={'NumSVCS': 'RemarkCodes'}, inplace=True)
        df.drop('SurfaceNumSvcs', axis=1, inplace=True)
    else:
        df.rename(columns={'SurfaceNumSvcs': 'l'}, inplace=True)
        
    t_list.append(df)

In [23]:
# Convert DF into exel
excel_file_path = 'C:\\ppaetna\\SD%20Payor%20Scraping/file1.xlsx'

t_list[0].to_excel(excel_file_path, index=False) 

excel_file_path = 'C:\\ppaetna\\SD%20Payor%20Scraping/file2.xlsx'
t_list[4].to_excel(excel_file_path, index=False) 


In [ ]:
dt = []
for df in t_list:
    for i in range(df.shape[0]-1):
        if df.iloc[i, 0] == '':
            col_index = df.columns.get_loc('ContractualObligations')
            
            df.iloc[i-1, col_index] = df.iloc[i-1,col_index] + ' ' + df.iloc[i,col_index]
            print(df.iloc[i-1, col_index])
            df = df.drop(i).reset_index(drop=True)
            i = 0
    dt.append(df)        
        
    
    

In [ ]:
dt[8]

In [ ]:
t_data_dict = []
count = 1
for t in t_list:
    t = t.to_dict("records")
    for i in t:
        i.update({'ActualAllowed':'', 'Enrollee_ClaimID':'', 'OtherAdjustments':'', 'PayerInitiatedReductions':'', 'Adjustments':''})

    print("T",t)
    print("\n")
    t_data_dict.append(t)
for d in t_data_dict:
#     print(count)
    count += 1
#     print(d)
    print("\n")

In [ ]:
eobclaimdetail = process_list(t_data_dict)
print(eobclaimdetail)